In [ ]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform

/Users/pavelmakarchuk/anaconda3/envs/pe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
reform = Reform.from_dict({
    "gov.simulation.labor_supply_responses.elasticities.income": {
        "2024-01-01.2100-12-31": -0.05
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.1": {
        "2024-01-01.2100-12-31": 0.31
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.2": {
        "2024-01-01.2100-12-31": 0.28
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.3": {
        "2024-01-01.2100-12-31": 0.27
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.4": {
        "2024-01-01.2100-12-31": 0.27
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.5": {
        "2024-01-01.2100-12-31": 0.25
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.6": {
        "2024-01-01.2100-12-31": 0.25
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.7": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.8": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.9": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.primary.10": {
        "2024-01-01.2100-12-31": 0.22
    },
    "gov.simulation.labor_supply_responses.elasticities.substitution.by_position_and_decile.secondary": {
        "2024-01-01.2100-12-31": 0.27
    },
},
country_id="us"
)

In [ ]:
baseline = Microsimulation()
reformed = Microsimulation(reform=reform)

In [ ]:
baseline_net_income = baseline.calculate("household_net_income")


In [ ]:
print(baseline_net_income)


               value        weight
0      142370.453125   4095.453369
1       57417.761719  27992.656250
2       43975.804688   1122.665283
3      130987.804688    882.152710
4      129959.000000  28653.367188
...              ...           ...
21103  105728.492188      0.131878
21104  106068.796875    445.853912
21105   72925.476562      0.064135
21106  294985.687500  10090.434570
21107   49588.898438      0.002831

[21108 rows x 2 columns]


In [ ]:
reformed_net_income = reformed.calculate("household_net_income")

Exception: RecursionError while calculating ma_tafdc_financial_eligible for period 2024-01. The full computation stack is:
  - household_net_income 2024, baseline_lsr_measurement
  - household_benefits 2024, baseline_lsr_measurement
  - snap 2024, baseline_lsr_measurement
  - snap 2024-01, baseline_lsr_measurement
  - snap_normal_allotment 2024-01, baseline_lsr_measurement
  - is_snap_eligible 2024-01, baseline_lsr_measurement
  - meets_snap_net_income_test 2024-01, baseline_lsr_measurement
  - snap_net_income_fpg_ratio 2024-01, baseline_lsr_measurement
  - snap_net_income 2024-01, baseline_lsr_measurement
  - snap_deductions 2024-01, baseline_lsr_measurement
  - snap_dependent_care_deduction 2024-01, baseline_lsr_measurement
  - childcare_expenses 2024-01, baseline_lsr_measurement
  - childcare_expenses 2024, baseline_lsr_measurement
  - ma_child_care_subsidies 2024, baseline_lsr_measurement
  - ma_ccfa 2024, baseline_lsr_measurement
  - ma_ccfa 2024-01, baseline_lsr_measurement
  - m

In [ ]:
print(reformed_net_income)